In [1]:
import eumdac
import datetime
import shutil
from tqdm import tqdm

In [2]:
consumer_key = '9dcHtMpRngCEnHBtZX4iOZL7s5Qa'
consumer_secret = 'FfJCsfRfPPOThUvOYfBEFkPzxCwa'
credentials = (consumer_key, consumer_secret)
token = eumdac.AccessToken(credentials)
print(f"This token '{token}' expires {token.expiration}")

This token 'b8facbfb-d9a7-33aa-9dbc-aa8089eac1db' expires 2024-06-17 09:50:47.173916


In [3]:
datastore = eumdac.DataStore(token)
for collection in datastore.collections:
    print(collection)

EO:EUM:DAT:MSG:CLM-IODC
EO:EUM:DAT:0394
EO:EUM:DAT:0412
EO:EUM:DAT:0857
EO:EUM:DAT:0862
EO:EUM:DAT:0882
EO:EUM:DAT:METOP:ASCSZF1B
EO:EUM:DAT:METOP:ASCSZR1B
EO:EUM:DAT:MSG:MSG15-RSS
EO:EUM:DAT:0855
EO:EUM:DAT:0836
EO:EUM:CM:METOP:ASCSZFR02
EO:EUM:DAT:0081
EO:EUM:DAT:0398
EO:EUM:DAT:0405
EO:EUM:DAT:0406
EO:EUM:DAT:0410
EO:EUM:DAT:0413
EO:EUM:DAT:0415
EO:EUM:DAT:0416
EO:EUM:DAT:0239
EO:EUM:DAT:0301
EO:EUM:DAT:0617
EO:EUM:DAT:0856
EO:EUM:DAT:METOP:ASCSZO1B
EO:EUM:DAT:METOP:GOMEL1
EO:EUM:DAT:0647
EO:EUM:DAT:0851
EO:EUM:DAT:0835
EO:EUM:DAT:0833
EO:EUM:DAT:0880
EO:EUM:DAT:0237
EO:EUM:DAT:0241
EO:EUM:DAT:0274
EO:EUM:DAT:0082
EO:EUM:DAT:0584
EO:EUM:DAT:MULT:HIRSL1
EO:EUM:DAT:METOP:GLB-SST-NC
EO:EUM:DAT:MSG:CLM
EO:EUM:DAT:0302
EO:EUM:DAT:0236
EO:EUM:DAT:0408
EO:EUM:DAT:0409
EO:EUM:DAT:0558
EO:EUM:DAT:METOP:OSI-150-A
EO:EUM:DAT:METOP:OSI-150-B
EO:EUM:DAT:MSG:RSS-CLM
EO:EUM:DAT:0615
EO:EUM:DAT:0858
EO:EUM:DAT:0859
EO:EUM:DAT:0839
EO:EUM:DAT:0841
EO:EUM:DAT:0834
EO:EUM:CM:METOP:ASCSZOR02
EO:EUM:CM:

In [4]:
selected_collection = datastore.get_collection('EO:EUM:DAT:0410')

In [9]:
# Add vertices for polygon, wrapping back to the start point.

min_lat = 55.5
max_lat = 64.5
min_lon = -30
max_lon = -15

geometry = [[min_lon, min_lat],[min_lon, max_lat],[max_lon, max_lat],[max_lon, min_lat],[min_lon, min_lat]]
# Set sensing start and end time
start = datetime.datetime(2024, 6, 16)
end = datetime.datetime(2024, 6, 17)
# Retrieve datasets that match our filter
products = selected_collection.search(
    geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in geometry])),
    dtstart=start, 
    dtend=end)
    
print(f'Found Datasets: {products.total_results} datasets for the given time range')
for product in products:
    print(str(product))


Found Datasets: 4 datasets for the given time range
S3A_OL_1_ERR____20240616T130743_20240616T135209_20240616T152804_2666_113_295______MAR_O_NR_002.SEN3
S3B_OL_1_ERR____20240616T122852_20240616T131318_20240616T144953_2666_094_152______MAR_O_NR_002.SEN3
S3A_OL_1_ERR____20240616T112644_20240616T121110_20240616T135146_2666_113_294______MAR_O_NR_002.SEN3
S3B_OL_1_ERR____20240616T104753_20240616T113219_20240616T131225_2666_094_151______MAR_O_NR_002.SEN3


In [10]:
# Print the number of datasets found and their details
print(f'Found Datasets: {products.total_results} datasets for the given time range')
for product in products:
    print(str(product))

# Download the data with a progress bar
for product in products:
    with product.open() as fsrc:
        # Get the total length of the file from the response headers
        total_length = int(fsrc.info().get('Content-Length', 0))

        # Open the destination file
        with open(fsrc.name, mode='wb') as fdst, \
             tqdm(total=total_length, unit='B', unit_scale=True, desc=f"Downloading {product}") as pbar:
            
            # Define a chunk size for reading the data
            chunk_size = 1024
            
            # Read the file in chunks and write to the destination file
            for chunk in iter(lambda: fsrc.read(chunk_size), b''):
                if chunk:  # filter out keep-alive new chunks
                    fdst.write(chunk)
                    pbar.update(len(chunk))
    
    print(f'Download of product {product} finished.')

print('All downloads are finished.')

Found Datasets: 4 datasets for the given time range
S3A_OL_1_ERR____20240616T130743_20240616T135209_20240616T152804_2666_113_295______MAR_O_NR_002.SEN3
S3B_OL_1_ERR____20240616T122852_20240616T131318_20240616T144953_2666_094_152______MAR_O_NR_002.SEN3
S3A_OL_1_ERR____20240616T112644_20240616T121110_20240616T135146_2666_113_294______MAR_O_NR_002.SEN3
S3B_OL_1_ERR____20240616T104753_20240616T113219_20240616T131225_2666_094_151______MAR_O_NR_002.SEN3


Download of product S3A_OL_1_ERR____20240616T130743_20240616T135209_20240616T152804_2666_113_295______MAR_O_NR_002.SEN3 finished.


Download of product S3B_OL_1_ERR____20240616T122852_20240616T131318_20240616T144953_2666_094_152______MAR_O_NR_002.SEN3 finished.


Download of product S3A_OL_1_ERR____20240616T112644_20240616T121110_20240616T135146_2666_113_294______MAR_O_NR_002.SEN3 finished.


Download of product S3B_OL_1_ERR____20240616T104753_20240616T113219_20240616T131225_2666_094_151______MAR_O_NR_002.SEN3 finished.
All downloads are finished.
